In [1]:
import pandas as pd
import numpy as np

In [2]:
# get a single long string of strain names separated by newline chars
with open('cat_names.csv') as f:
    file = f.read()

In [3]:
import random
import string
import re

all_characters = string.printable
n_characters = len(all_characters)

In [4]:
chunk_len = 100
file_len = len(file)
def random_chunk():
    '''filelen is length of chars in file'''
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

In [5]:
# Build the Model
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        input = self.encoder(input.view(1,-1))
        output, hidden = self.gru(input.view(1,1,-1) ,hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))



In [6]:
def char_tensor(string_):
    tensor = torch.zeros(len(string_)).long()
    for c in range(len(string_)):
        tensor[c] = all_characters.index(string_[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

Variable containing:
 10
 11
 12
 39
 40
 41
[torch.LongTensor of size 6]



In [7]:
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target


In [8]:
# Evaluating
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str
    
    for p in range(len(prime_str)-1):
        _,hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)
        
    return predicted

In [9]:
# Training
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m,s)

In [10]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss=0
    
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c])
        
    loss.backward()
    decoder_optimizer.step()
    
    return loss.data[0] / chunk_len

In [11]:
n_epochs = 2000
print_every = 100
plot_every=10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses=[]
loss_avg = 0

for epoch in range(1, n_epochs+1):
    try:
        loss = train(*random_training_set())
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
            print(evaluate('R', 100), '\n')

        if epoch % plot_every == 0:
            all_losses.append(loss_avg / plot_every)
            loss_avg = 0
    except:
        continue

 13s (100 5%) 2.4004]
RpRe
Cacbebt
Maslie Bela
Bele
Torgioo
Omdie
Telta
Kuer
Dia
Tager
Belew Frroele
Omino
Trujiga
Boow
Fes 

 27s (200 10%) 2.3936]
Ra Bora
Syore
Zacy
Omi Sani
Eliro
Hoo
Hotty
Felie
Koon
Sesterolon
Geander
Mrano
Grarue
Bosten
Secy
Pr 

 40s (300 15%) 2.3351]
Ragdy
Sabu
Mald
Tosppin
Lalc
Loxin
Abnan Doard
Malli
Fren
Alla
Liger
Chortorla
Dassie
Hargie
Oliea
Ch 

 55s (400 20%) 2.5958]
Ron
Dackrie
Cack
Jink
Poppy
Ild
Copadro
Wumber Tumi
Missas
Ney
Sperat
Hoze
Moja
Molly
Jack
Molly
Alic 

 8s (500 25%) 2.2254]
Rufla
Rilly
Kayty
Jaxget
Samale
Jem
Dellan
Stleris
Mixie
Lek
Lilo
Miny
Dennin
Ever
Zee
Nipollda
Tigge 

 24s (600 30%) 2.3733]
Rope
Sade
Bubbby
Leeki
Ozzie
Sherace
Olive
Mata
Burter
Kidna
Freusty
Sluggin
Tasie
Lokie
Cherleted
Ki 

 38s (700 35%) 2.0252]
Run
Pricem
Missy
Abby
Luva
Woo
Frudley
Laisa
Mucks
Moggy
Ayalla
Chani
Ala
Golf
Jamh
Finie
Kriper
Than 

 52s (800 40%) 2.0167]
Riso
Deisha
Miss Dimbus
Bloe
Simmin
Banie
Caster
Kiki
Lily
Carson
Jenne
Susty
Mia
Blacky
B

In [15]:
print(evaluate('f', 100, temperature=0.5))

fill
Lulu
Pronie
Stella
Pepper
Rocky
Parro
Bella
Patch
Samiley
Stella
Sammi
Suokie
Sanda
Sam
Linz
Pip


In [22]:
print(evaluate('r', 100, temperature=0.8))

raw
Savia
Sansey
Sammet Ceramin
Erlo
Merro
Bartch
Potty
Seanie
Isaby
Spother
Mink
Gracie
Cetty
Bean
A


In [33]:
print(evaluate('c', 200, temperature=0.8)) 

ck
Bennur
Habet
Poppy
Blooa
Hawballea
Zoe
Luo
Mingian
Dalsie Samine
Snow
Sander
Maspy
Minkey
Sam
Zerinora
Zoosh
Vicrina
Blue
Marple
Hanuter
Fio
Annie
Cachur
Ravin
Landy
Pappyn
Mandy
Linceld
Lucy
Theo
I


In [ ]:
# ...So what happens if we input unique names -- I didn't take out names